# Barcode Detector
___

Here we will demonstrate a basic implementation of barcode detection using computer vision and image processing techniques. Although this code won't work on all barcodes and the accuracy of detecting barcodes might be low, it will surely give you a ffair intuition on what steps should be followed to make a barcode detector.

In [ ]:
# import the necessary packages
import numpy as np
import argparse
import cv2

# load the image and convert it to grayscale
image = cv2.imread("images/barcode_02.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# compute the Scharr gradient magnitude representation of the images
# in both the x and y direction
gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)

# subtract the y-gradient from the x-gradient
gradient = cv2.subtract(gradX, gradY)
gradient = cv2.convertScaleAbs(gradient)

# blur and threshold the image
blurred = cv2.blur(gradient, (9, 9))
(_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

# construct a closing kernel and apply it to the thresholded image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# perform a series of erosions and dilations
closed = cv2.erode(closed, None, iterations = 4)
closed = cv2.dilate(closed, None, iterations = 4)

# find the contours in the thresholded image, then sort the contours
# by their area, keeping only the largest one
(_,cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]

# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

# draw a bounding box arounded the detected barcode and display the
# image
cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
cv2.imshow("Image", image)
cv2.waitKey(0)

As always we import the required modules, load our image  and convert it to grayscale.

Then, we use the Scharr operator (specified using ksize = -1 ) to construct the gradient magnitude representation of the grayscale image in the horizontal and vertical directions .

From there, we subtract the y-gradient of the Scharr operator from the x-gradient of the Scharr operator. By performing this subtraction we are left with regions of the image that have high horizontal gradients and low vertical gradients.
Our gradient representation of our original image above looks like:

<img src="captures/gradient.png" height = 415px width = 330px style="float:left; margin-left:30px">

Notice how the barcoded region of the image has been detected by our gradient operations. The next steps will be to filter out the noise in the image and focus solely on the barcode region.

The first thing we’ll do is apply an average blur to the gradient image using a 9 x 9 kernel. This will help smooth out high frequency noise in the gradient representation of the image. We’ll then threshold the blurred image. Any pixel in the gradient image that is not greater than 225 is set to 0 (black). Otherwise, the pixel is set to 255 (white).

The output of the blurring and thresholding looks like this:

<img src="captures/bthresh.png" height = 415px width = 330px style="float:left; margin-left:30px">

However, as you can see in the threshold image above, there are gaps between the vertical bars of the barcode. In order to close these gaps and make it easier for our algorithm to detect the “blob”-like region of the barcode, we’ll performed basic morphological operations. We constructedg a rectangular kernel using the cv2.getStructuringElement. This kernel has a width that is larger than the height, thus allowing us to close the gaps between vertical stripes of the barcode.

We then perform our morphological operation by applying our kernel to our thresholded image, thus attempting to close the the gaps between the bars. to remove these small blobs, that are not part of the actual barcode, but may interfere with our contour detection, we performed 4 iterations of erosions, followed by 4 iterations of dilations. An erosion will “erode” the white pixels in the image, thus removing the small blobs, whereas a dilation will “dilate” the remaining white pixels and grow the white regions back out. Provided that the small blobs were removed during the erosion, they will not reappear during the dilation.

After our series of erosions and dilations you can see that the small blobs have been successfully removed and we are left with the barcode region:

<img src="captures/closing.png" height = 415px width = 330px style="float:left; margin-left:30px">

Then ultimately we find the contours of the barcoded region of the image. We simply find the largest contour in the image, should correspond to the barcoded region. We then determine the minimum bounding box for the largest contour and finally display the detected barcode .

As you can see in the following image, we have successfully detected the barcode:

<img src="captures/barcodedetection.png" height = 500px width = 440px style="float:left; margin-left:30px">

___